![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Legal Data Augmentation with Chunk Mappers

In [0]:
from johnsnowlabs import *

# About Data Augmentation

__Data Augmentation__ is the process of increase an extracted datapoint with external sources. 

For example, let's suppose I work with a document which mentions the company _Apple_. We can extract that entity using using NER.

But we can do much more than that! Public companies have a lot of information published in the Internet about them. We can check legal and financial information from those companies in Legal NLP, in an offline-mode!

With __Chunk Mappers__, we can use external sources, as _SEC Edgar, Nasdaq_ or even _Wikidata_, to enrich `Apple` with much more information, allowing us to take better decisions.

Let's see how to do it.

# Step 1: Name Entity Recognition

Let's suppose we get this news from scrapping the Internet, or from Wikipedia.

In [0]:
text = """APPLE, INC. became the first publicly traded U.S. company to be valued at over $1 trillion in August 2018, then $2 trillion in August 2020, and most recently $3 trillion in January 2022. """

text

Out[8]: 'APPLE, INC. became the first publicly traded U.S. company to be valued at over $1 trillion in August 2018, then $2 trillion in August 2020, and most recently $3 trillion in January 2022. '

Firstly, We use NER model to extract the companies name from the text.

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

ner_model = legal.NerModel.pretrained("legner_orgs_prods_alias", "en", "legal/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")
        
ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

nlpPipeline = nlp.Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter,
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ][OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]
legner_orgs_prods_alias download started this may take some time.
[ | ][OK!]

## We use LightPipelines to get the result

In [0]:
lp_ner = nlp.LightPipeline(model)

ner_result = lp_ner.fullAnnotate(text)

In [0]:
import pandas as pd

chunks = []
entities = []
begin = []
end = []

for n in ner_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    
df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'entities':entities})

df.head(20)

Out[11]:

,chunks,begin,end,entities
0,"APPLE, INC.",0,10,ORG


Alright! Company names has been detected as an organization.

# Step 2: Mapping NER to External Data Using Chunk Mappers (offline)

Very often, the name of the organizations we find in texts are not their official name. For example, if we train to find `APPLE, INC` in SEC Edgar, we won't find it as it is.

Every data provider may have different versions of the official names. Most of them will include organization types as `Inc`, `Corp` etc. But again, Some others may have `Inc.`, `Incorporated`, etc.

**ChunkMappers** by default work with exact matches, so before being able to map our ORG detected by NER to Chunk Mappers, we need to:
1. Either enable Fuzzy Matching in Chunk Mappers (Section 2a);
2. Or normalize the company name with Entity Resolvers (Section 2n);

![my_test_image](files/Screenshot_2023_02_24_152350.png)

Let's suppose we want to manually get information about these companies.

Since it's a public US company, we can go to [SEC Edgar's database](https://www.sec.gov/edgar/searchedgar/companysearch) and look for it.

THere is no `APPLE, INC.` in Edgar, but another variation of it (`APPLE INC`). Let's check the two methods we can apply to do the mapping from one to another.

## Step 2a) ChunkMappers Fuzzy Matching

Let's get several variations of Apple to see how we can use Fuzzy Matching to get the official name of Apple in Edgar.

In [0]:
ORG = df['chunks'].tolist()

ORG

Out[32]: ['APPLE, INC.']

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

# Posible distance metrics: ['levenshtein', 'longest-common-subsequence', 'cosine']
CM = legal.ChunkMapperModel().pretrained("legmapper_edgar_companyname", "en", "legal/models")\
    .setInputCols(["document"])\
    .setOutputCol("mappings")\
    .setEnableFuzzyMatching(True)\
    .setEnableCharFingerprintMatching(False)\
    .setFuzzyMatchingDistances(['cosine'])\
    .setFuzzyMatchingDistanceThresholds([5])

cm_pipeline = nlp.Pipeline(stages=[document_assembler, CM])

empty_data = spark.createDataFrame([[""]]).toDF("text")

fit_cm_pipeline = cm_pipeline.fit(empty_data)

lp = nlp.LightPipeline(fit_cm_pipeline)

res = lp.fullAnnotate(ORG)

legmapper_edgar_companyname download started this may take some time.
[ | ][OK!]

In [0]:
for r in res:
  for map in r['mappings']:
    print(map)
  print('\n')

Annotation(labeled_dependency, 0, 10, BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1, {'__trained__': 'BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1', 'relation': 'name', '__distance_function__': 'jaccard', 'ops': '0.8285014148574912', 'all_relations': '', 'entity': 'APPLE, INC.', 'distance': '0.631578947368421', '__relation_name__': 'name'}, [])
Annotation(labeled_dependency, 0, 10, [6189, {'__trained__': 'BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1', 'relation': 'sic', '__distance_function__': 'jaccard', 'ops': '0.8285014148574912', 'all_relations': '', 'entity': 'APPLE, INC.', 'distance': '0.631578947368421', '__relation_name__': 'sic'}, [])
Annotation(labeled_dependency, 0, 10, 6189, {'__trained__': 'BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1', 'relation': 'sic_code', '__distance_function__': 'jaccard', 'ops': '0.8285014148574912', 'all_relations': '', 'entity': 'APPLE, INC.', 'distance': '0.631578947368421', '__relation_name__': 'sic_code'}, [])
Annotation(labeled_dependency, 0, 10, 0, {'__trained__': 'BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1', 'relation': 'irs_number', '__distance_function__': 'jaccard', 'ops': '0.8285014148574912', 'all_relations': '', 'entity': 'APPLE, INC.', 'distance': '0.631578947368421', '__relation_name__': 'irs_number'}, [])
Annotation(labeled_dependency, 0, 10, 1231, {'__trained__': 'BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1', 'relation': 'fiscal_year_end', '__distance_function__': 'jaccard', 'ops': '0.8285014148574912', 'all_relations': '', 'entity': 'APPLE, INC.', 'distance': '0.631578947368421', '__relation_name__': 'fiscal_year_end'}, [])
Annotation(labeled_dependency, 0, 10, NY, {'__trained__': 'BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1', 'relation': 'state_location', '__distance_function__': 'jaccard', 'ops': '0.8285014148574912', 'all_relations': '', 'entity': 'APPLE, INC.', 'distance': '0.631578947368421', '__relation_name__': 'state_location'}, [])
Annotation(labeled_dependency, 0, 10, DE, {'__trained__': 'BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1', 'relation': 'state_incorporation', '__distance_function__': 'jaccard', 'ops': '0.8285014148574912', 'all_relations': '', 'entity': 'APPLE, INC.', 'distance': '0.631578947368421', '__relation_name__': 'state_incorporation'}, [])
Annotation(labeled_dependency, 0, 10, 245 PARK AVENUE, {'__trained__': 'BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1', 'relation': 'business_street', '__distance_function__': 'jaccard', 'ops': '0.8285014148574912', 'all_relations': '', 'entity': 'APPLE, INC.', 'distance': '0.631578947368421', '__relation_name__': 'business_street'}, [])
Annotation(labeled_dependency, 0, 10, NEW YORK, {'__trained__': 'BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1', 'relation': 'business_city', '__distance_function__': 'jaccard', 'ops': '0.8285014148574912', 'all_relations': '', 'entity': 'APPLE, INC.', 'distance': '0.631578947368421', '__relation_name__': 'business_city'}, [])
Annotation(labeled_dependency, 0, 10, NY, {'__trained__': 'BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1', 'relation': 'business_state', '__distance_function__': 'jaccard', 'ops': '0.8285014148574912', 'all_relations': '', 'entity': 'APPLE, INC.', 'distance': '0.631578947368421', '__relation_name__': 'business_state'}, [])
Annotation(labeled_dependency, 0, 10, 10167, {'__trained__': 'BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1', 'relation': 'business_zip', '__distance_function__': 'jaccard', 'ops': '0.8285014148574912', 'all_relations': '', 'entity': 'APPLE, INC.', 'distance': '0.631578947368421', '__relation_name__': 'business_zip'}, [])
Annotation(labeled_dependency, 0, 10, , {'__trained__': 'BEAR STEARNS ASSET BAC SE INC MOR BA LIB NOT CL A SE 1999-1', 'relation': 'business_phone', '__distance_function__': 'jaccard', 'ops': '0.8285014148574912', 'all_relations': '', 'enti

**We have been able to successfully retrieve the information in Edgar using different variations of the Company Name `Apple`!**

## Step 2b: Using Entity Resolvers for Company Names Normalization

`Company Name Normalization` is the process of obtaining the name of the company used by data providers, usually the **"official"** name of the company.

Sometimes, some data providers may have different versions of the name with different punctuation. For example, for Meta:
- Meta Platforms, Inc.
- Meta Platforms Inc.
- Meta Platforms, Inc
- etc

So, it's mandatory we do `Company Normalization` taking into account the database / datasource provider we want to extract data from. The data providers we have are:
- SEC Edgar
- Wikidata
- etc.

Let's normalize `APPLE INC` to the official name in _SEC Edgar_.

In [0]:
embeddings = nlp.UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")
    
resolver = legal.SentenceEntityResolverModel.pretrained("legel_edgar_company_name", "en", "legal/models")\
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("resolution")\
      .setDistanceFunction("EUCLIDEAN")

pipelineModel = nlp.PipelineModel(
      stages = [
          documentAssembler,
          embeddings,
          resolver])

lp_res = nlp.LightPipeline(pipelineModel)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
legel_edgar_company_name download started this may take some time.
[ | ][OK!]

In [0]:
ORG = df['chunks'].tolist()

ORG

Out[36]: ['APPLE, INC.']

In [0]:
el_res = lp_res.annotate(ORG)

el_res

Out[37]: [{'document': ['APPLE, INC.'],
 'sentence_embeddings': ['APPLE, INC.'],
 'resolution': ['APPLE INC']}]

Here is our normalized name for:
- Apple: `APPLE INC`.

Now, let's see which information is available in Edgar database for `APPLE INC` company

In [0]:
NORM_ORG = el_res[0]["resolution"]

NORM_ORG

Out[38]: ['APPLE INC']

## And now, we do exact match with the normalized version

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

# Posible distance metrics: ['levenshtein', 'longest-common-subsequence', 'cosine']
CM = legal.ChunkMapperModel().pretrained("legmapper_edgar_companyname", "en", "legal/models")\
    .setInputCols(["document"])\
    .setOutputCol("mappings")\

cm_pipeline = nlp.Pipeline(stages=[document_assembler, CM])

empty_data = spark.createDataFrame([[""]]).toDF("text")

fit_cm_pipeline = cm_pipeline.fit(empty_data)

lp = nlp.LightPipeline(fit_cm_pipeline)

res = lp.fullAnnotate(NORM_ORG)

legmapper_edgar_companyname download started this may take some time.
[ | ][OK!]

In [0]:
for r in res:
  for map in r['mappings']:
    print(map)

Annotation(labeled_dependency, 0, 8, APPLE INC, {'__trained__': 'APPLE INC', 'relation': 'name', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'name'}, [])
Annotation(labeled_dependency, 0, 8, ELECTRONIC COMPUTERS [3571], {'__trained__': 'APPLE INC', 'relation': 'sic', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'sic'}, [])
Annotation(labeled_dependency, 0, 8, 3571, {'__trained__': 'APPLE INC', 'relation': 'sic_code', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '0', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'sic_code'}, [])
Annotation(labeled_dependency, 0, 8, 942404110, {'__trained__': 'APPLE INC', 'relation': 'irs_number', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '0', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'irs_number'}, [])
Annotation(labeled_dependency, 0, 8, 930, {'__trained__': 'APPLE INC', 'relation': 'fiscal_year_end', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '924:::927:::0:::929:::925:::926', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'fiscal_year_end'}, [])
Annotation(labeled_dependency, 0, 8, CA, {'__trained__': 'APPLE INC', 'relation': 'state_location', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'state_location'}, [])
Annotation(labeled_dependency, 0, 8, CA, {'__trained__': 'APPLE INC', 'relation': 'state_incorporation', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'state_incorporation'}, [])
Annotation(labeled_dependency, 0, 8, ONE INFINITE LOOP, {'__trained__': 'APPLE INC', 'relation': 'business_street', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': 'ONE APPLE PARK WAY:::1 INFINITE LOOP', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'business_street'}, [])
Annotation(labeled_dependency, 0, 8, CUPERTINO, {'__trained__': 'APPLE INC', 'relation': 'business_city', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'business_city'}, [])
Annotation(labeled_dependency, 0, 8, CA, {'__trained__': 'APPLE INC', 'relation': 'business_state', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'business_state'}, [])
Annotation(labeled_dependency, 0, 8, 95014, {'__trained__': 'APPLE INC', 'relation': 'business_zip', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'business_zip'}, [])
Annotation(labeled_dependency, 0, 8, (408) 996-1010, {'__trained__': 'APPLE INC', 'relation': 'business_phone', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '4089961010', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'business_phone'}, [])
Annotation(labeled_dependency, 0, 8, APPLE COMPUTER INC, {'__trained__': 'APPLE INC', 'relation': 'former_name', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'former_name'}, [])
Annotation(labeled_dependency, 0, 8, 19970808, {'__trained__': 'APPLE INC', 'relation': 'former_name_date', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '', 'entity': 'APPLE INC', 'distance': '0.0', '__relation_name__': 'former_name_date'}, [])
Annotation(labeled_dependency, 0, 8, 2017-02-01, {'__trained__': 'APPLE INC', 'relation': 'date', '__distance_function__': 'levenshtein', 'ops': '0.0', 'all_relations': '2017-11-03:::2016-01-27:::2016-07-27:::2016-10-26:::

# Train Your Own ChunkMapper Model

Here, we will train a ChunkMapper model with 1000 samples

### Load Dataset

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/legal-nlp/data/sample_openedgar.json

dbutils.fs.cp("file:/databricks/driver/sample_openedgar.json", "dbfs:/") 

Out[43]: True

In [0]:
import json
with open('sample_openedgar.json', 'r') as f:
    company_json = json.load(f)

In [0]:
company_json['mappings'][1]

Out[45]: {'key': 'StepOne Personal Health, Inc.',
 'relations': [{'key': 'name', 'values': ['StepOne Personal Health, Inc.']},
 {'key': 'sic',
 'values': ['SERVICES-COMPUTER PROCESSING & DATA PREPARATION [7374]']},
 {'key': 'sic_code', 'values': [7374, 0]},
 {'key': 'irs_number', 'values': [900785095, 0]},
 {'key': 'fiscal_year_end', 'values': [1231, 0]},
 {'key': 'state_location', 'values': ['MO', 'CA']},
 {'key': 'state_incorporation', 'values': ['DE']},
 {'key': 'business_street',
 'values': ['509 SOUGH WALL AVENUE', 'C/O: ALTERNATIVE SECURITIES MARKET']},
 {'key': 'business_city', 'values': ['JOPLIN', 'BEVERLY HILLS']},
 {'key': 'business_state', 'values': ['MO', 'CA']},
 {'key': 'business_zip', 'values': ['64801', '90210']},
 {'key': 'business_phone', 'values': ['650-454-0862', '213-407-4386']},
 {'key': 'former_name', 'values': ['']},
 {'key': 'former_name_date', 'values': ['']},
 {'key': 'date',
 'values': ['2017-01-24',
 '2015-03-23',
 '2015-08-18',
 '2015-08-25',
 '2015-10-13',
 '2015-10-16',
 '2015-10-29',
 '2015-10-30',
 '2015-12-10',
 '2015-12-22']},
 {'key': 'company_id', 'values': [1606811]}]}

### Check a sample company

In [0]:
for x in company_json['mappings']:
    if 'StepOne Personal Health, Inc.' in x['key']:
        print(x)

{'key': 'StepOne Personal Health, Inc.', 'relations': [{'key': 'name', 'values': ['StepOne Personal Health, Inc.']}, {'key': 'sic', 'values': ['SERVICES-COMPUTER PROCESSING & DATA PREPARATION [7374]']}, {'key': 'sic_code', 'values': [7374, 0]}, {'key': 'irs_number', 'values': [900785095, 0]}, {'key': 'fiscal_year_end', 'values': [1231, 0]}, {'key': 'state_location', 'values': ['MO', 'CA']}, {'key': 'state_incorporation', 'values': ['DE']}, {'key': 'business_street', 'values': ['509 SOUGH WALL AVENUE', 'C/O: ALTERNATIVE SECURITIES MARKET']}, {'key': 'business_city', 'values': ['JOPLIN', 'BEVERLY HILLS']}, {'key': 'business_state', 'values': ['MO', 'CA']}, {'key': 'business_zip', 'values': ['64801', '90210']}, {'key': 'business_phone', 'values': ['650-454-0862', '213-407-4386']}, {'key': 'former_name', 'values': ['']}, {'key': 'former_name_date', 'values': ['']}, {'key': 'date', 'values': ['2017-01-24', '2015-03-23', '2015-08-18', '2015-08-25', '2015-10-13', '2015-10-16', '2015-10-29', '2015-10-30', '2015-12-10', '2015-12-22']}, {'key': 'company_id', 'values': [1606811]}]}

### Check all keys

In [0]:
all_rels = [x['key'] for x in company_json['mappings'][0]['relations']]

In [0]:
all_rels

Out[48]: ['name',
 'sic',
 'sic_code',
 'irs_number',
 'fiscal_year_end',
 'state_location',
 'state_incorporation',
 'business_street',
 'business_city',
 'business_state',
 'business_zip',
 'business_phone',
 'former_name',
 'former_name_date',
 'date',
 'company_id']

### Create ChunkMapperApproach

In [0]:
chunkerMapper = legal.ChunkMapperApproach()\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("mappings")\
      .setDictionary("dbfs:/sample_openedgar.json")\
      .setRels(all_rels)

In [0]:
empty_dataset = spark.createDataFrame([[""]]).toDF("text")

In [0]:
fit_CM = chunkerMapper.fit(empty_dataset)

In [0]:
# Save model
fit_CM.write().overwrite().save('/dbfs/openedgar_2000_2022_company_mapper')

### Let's test our ChunkMapper model

In [0]:
text = ["""StepOne Personal Health, Inc. is an American solar cell and engineered wafer manufacturer."""]

In [0]:
# We get company name from sample text

ner_result = lp_ner.fullAnnotate(text)

ner_result

Out[71]: [{'document': [Annotation(document, 0, 89, StepOne Personal Health, Inc. is an American solar cell and engineered wafer manufacturer., {}, [])],
 'ner_chunk': [Annotation(chunk, 0, 28, StepOne Personal Health, Inc., {'entity': 'ORG', 'sentence': '0', 'chunk': '0', 'confidence': '0.60824996'}, [])],
 'token': [Annotation(token, 0, 6, StepOne, {'sentence': '0'}, []),
 Annotation(token, 8, 15, Personal, {'sentence': '0'}, []),
 Annotation(token, 17, 22, Health, {'sentence': '0'}, []),
 Annotation(token, 23, 23, ,, {'sentence': '0'}, []),
 Annotation(token, 25, 27, Inc, {'sentence': '0'}, []),
 Annotation(token, 28, 28, ., {'sentence': '0'}, []),
 Annotation(token, 30, 31, is, {'sentence': '0'}, []),
 Annotation(token, 33, 34, an, {'sentence': '0'}, []),
 Annotation(token, 36, 43, American, {'sentence': '0'}, []),
 Annotation(token, 45, 49, solar, {'sentence': '0'}, []),
 Annotation(token, 51, 54, cell, {'sentence': '0'}, []),
 Annotation(token, 56, 58, and, {'sentence': '0'}, []),
 Annotation(token, 60, 69, engineered, {'sentence': '0'}, []),
 Annotation(token, 71, 75, wafer, {'sentence': '0'}, []),
 Annotation(token, 77, 88, manufacturer, {'sentence': '0'}, []),
 Annotation(token, 89, 89, ., {'sentence': '0'}, [])],
 'ner': [Annotation(named_entity, 0, 6, B-ORG, {'word': 'StepOne', 'confidence': '0.8399', 'sentence': '0'}, []),
 Annotation(named_entity, 8, 15, I-ORG, {'word': 'Personal', 'confidence': '0.3797', 'sentence': '0'}, []),
 Annotation(named_entity, 17, 22, I-ORG, {'word': 'Health', 'confidence': '0.4497', 'sentence': '0'}, []),
 Annotation(named_entity, 23, 23, I-ORG, {'word': ',', 'confidence': '0.5961', 'sentence': '0'}, []),
 Annotation(named_entity, 25, 27, I-ORG, {'word': 'Inc', 'confidence': '0.8942', 'sentence': '0'}, []),
 Annotation(named_entity, 28, 28, I-ORG, {'word': '.', 'confidence': '0.4899', 'sentence': '0'}, []),
 Annotation(named_entity, 30, 31, O, {'word': 'is', 'confidence': '1.0', 'sentence': '0'}, []),
 Annotation(named_entity, 33, 34, O, {'word': 'an', 'confidence': '0.9999', 'sentence': '0'}, []),
 Annotation(named_entity, 36, 43, O, {'word': 'American', 'confidence': '0.9057', 'sentence': '0'}, []),
 Annotation(named_entity, 45, 49, O, {'word': 'solar', 'confidence': '0.7765', 'sentence': '0'}, []),
 Annotation(named_entity, 51, 54, O, {'word': 'cell', 'confidence': '0.382', 'sentence': '0'}, []),
 Annotation(named_entity, 56, 58, O, {'word': 'and', 'confidence': '0.9702', 'sentence': '0'}, []),
 Annotation(named_entity, 60, 69, O, {'word': 'engineered', 'confidence': '0.9782', 'sentence': '0'}, []),
 Annotation(named_entity, 71, 75, O, {'word': 'wafer', 'confidence': '0.6673', 'sentence': '0'}, []),
 Annotation(named_entity, 77, 88, O, {'word': 'manufacturer', 'confidence': '0.8334', 'sentence': '0'}, []),
 Annotation(named_entity, 89, 89, O, {'word': '.', 'confidence': '1.0', 'sentence': '0'}, [])],
 'embeddings': [Annotation(word_embeddings, 0, 6, StepOne, {'isOOV': 'false', 'pieceId': '101', 'isWordStart': 'true', 'token': 'StepOne', 'sentence': '0'}, []),
 Annotation(word_embeddings, 8, 15, Personal, {'isOOV': 'false', 'pieceId': '101', 'isWordStart': 'true', 'token': 'Personal', 'sentence': '0'}, []),
 Annotation(word_embeddings, 17, 22, Health, {'isOOV': 'false', 'pieceId': '101', 'isWordStart': 'true', 'token': 'Health', 'sentence': '0'}, []),
 Annotation(word_embeddings, 23, 23, ,, {'isOOV': 'false', 'pieceId': '116', 'isWordStart': 'true', 'token': ',', 'sentence': '0'}, []),
 Annotation(word_embeddings, 25, 27, Inc, {'isOOV': 'false', 'pieceId': '101', 'isWordStart': 'true', 'token': 'Inc', 'sentence': '0'}, []),
 Annotation(word_embeddings, 28, 28, ., {'isOOV': 'false', 'pieceId': '118', 'isWordStart': 'true', 'token': '.', 'sentence': '0'}, []),
 Annotation(word_embeddings, 30, 31, is, {'isOOV': 'false', 'pieceId': '686', 'isWordStart': 'true', 'token': 'is', 'sentence': '0'}, []),
 Annotation(word_embeddings, 33, 34, an, {'isOOV': 'false', 'pieceId': '620', 'isWordStart'

In [0]:
ORG = ner_result[0]["ner_chunk"][0].result

ORG

Out[72]: 'StepOne Personal Health, Inc.'

In [0]:
# We normalize company name

el_res = lp_res.annotate(ORG)

el_res

Out[73]: {'document': ['StepOne Personal Health, Inc.'],
 'sentence_embeddings': ['StepOne Personal Health, Inc.'],
 'resolution': ['StepOne Personal Health Inc']}

In [0]:
NORM_ORG = el_res["resolution"]

NORM_ORG

Out[74]: ['StepOne Personal Health Inc']

### Let's load our ChunkMapper model

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

chunkAssembler = nlp.Doc2Chunk() \
    .setInputCols("document") \
    .setOutputCol("chunk") \
    .setIsArray(False)

CM =legal.ChunkMapperModel().load("/dbfs/openedgar_2000_2022_company_mapper")\
      .setInputCols(["chunk"])\
      .setOutputCol("mappings")

cm_pipeline = nlp.Pipeline(stages=[documentAssembler, 
                                   chunkAssembler, 
                                   CM])

fit_cm_pipeline = cm_pipeline.fit(empty_data)

In [0]:
# LightPipelines don't support Doc2Chunk, so we will use here usual transform

df = spark.createDataFrame([NORM_ORG]).toDF("text")

df.show()

+--------------------+
 text|
+--------------------+
StepOne Personal ...|
+--------------------+

In [0]:
res = fit_cm_pipeline.transform(df)

res.show()

+--------------------+--------------------+--------------------+--------------------+
 text| document| chunk| mappings|
+--------------------+--------------------+--------------------+--------------------+
StepOne Personal ...|[{document, 0, 26...|[{chunk, 0, 26, S...|[{labeled_depende...|
+--------------------+--------------------+--------------------+--------------------+

In [0]:
res.select("mappings.result").show(truncate=False)

+------+
result|
+------+
[NONE]|
+------+

In [0]:
r = res.select("mappings").collect()
r

Out[79]: [Row(mappings=[Row(annotatorType='labeled_dependency', begin=0, end=26, result='NONE', metadata={'sentence': '0', 'chunk': '0', 'entity': 'StepOne Personal Health Inc'}, embeddings=[])])]